In [1]:
import torch
from fpn import FPN
from resnet import resnest50
from collections import OrderedDict

In [2]:
model = resnest50()
model

ResNet(
  (stem): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
  )
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): SplAtConv2d(
        (conv): Conv2d(6

In [3]:
fpn = FPN(in_channels=[256, 512, 1024, 2048], out_channels=256, num_outs=4)
fpn

FPN(
  (lateral_convs): ModuleList(
    (0): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (1): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (2): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (3): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
  )
  (fpn_conv): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)

In [14]:
pretrain_model = torch.load("mask_rcnn_s50_fpn_syncbn-backbone+head_mstrain_1x_coco_20200926_125503-8a2c3d47.pth")

#处理backbone
new_backbone_statedict = OrderedDict()
for key in pretrain_model['state_dict']:
    if key.startswith('backbone'):
        new_backbone_statedict[key[len("backbone."):]] = pretrain_model['state_dict'][key]
torch.save(new_backbone_statedict, "coco_backbone_statedict.pth")
model.load_state_dict(new_backbone_statedict)
del new_backbone_statedict

#处理fpn
temp_fpn_statedict = OrderedDict()
for key in pretrain_model['state_dict']:
    if not key.startswith("neck"):
        continue
    temp_fpn_statedict[key[len("neck."):]] = pretrain_model['state_dict'][key]
for i in range(1, 4):
    del temp_fpn_statedict['fpn_convs.{}.conv.weight'.format(i)]
    del temp_fpn_statedict['fpn_convs.{}.conv.bias'.format(i)]
new_fpn_statedict = OrderedDict()
for new_key, old_key in zip(fpn.state_dict(), temp_fpn_statedict):
    new_fpn_statedict[new_key] = temp_fpn_statedict[old_key]
del temp_fpn_statedict
torch.save(new_fpn_statedict, "coco_fpn_statedict.pth")
fpn.load_state_dict(new_fpn_statedict)
del new_fpn_statedict

In [15]:
img = torch.randn(2, 3, 512, 512)
outs = model(img)
for out in outs:
    print(out.shape)

torch.Size([2, 256, 128, 128])
torch.Size([2, 512, 64, 64])
torch.Size([2, 1024, 32, 32])
torch.Size([2, 2048, 16, 16])


In [17]:
fpn_outs = fpn(outs)
print(fpn_outs.shape)

torch.Size([2, 256, 128, 128])


In [2]:
import torch
from resnest50_FPN import HuBMAP_model
model = HuBMAP_model(
    num_class = 1,
    use_sigmoid=True,
    model_path="./resnest50_fpn_coco.pth"
)
img = torch.randn(2, 3, 512, 512)
pred = model(img)
print(pred.shape)

torch.Size([2, 1, 128, 128])


In [1]:
import torch
from model import HuBMAP_model

In [2]:
model = HuBMAP_model(
    num_class = 1,
    use_sigmoid=True,
    model_path="./预训练权重/resnest50_fpn_coco.pth"
)
model = model.cuda()
img = torch.randn(24, 3, 416, 416).cuda()
pred = model(img)
print(pred.shape)

torch.Size([24, 1, 416, 416])


In [ ]:
model = HuBMAP_model(
    num_class = 1,
    use_sigmoid=True,
    model_path="./预训练权重/resnest50_fpn_coco.pth"
)
img = torch.randn(2, 3, 512, 512)
pred = model(img)
print(pred.shape)

In [3]:
580*18

10440

In [3]:
(10060-785)//16

580

In [3]:
(7603-1282)//10

632

In [ ]:
import torchvision
model = torchvision.models.segmentation.fcn_resnet50(False)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to C:\Users\Admin/.cache\torch\checkpoints\resnet50-19c8e357.pth


In [1]:
import torch
import torchvision
import torch.nn as nn

In [2]:
model = torchvision.models.segmentation.fcn_resnet50(pretrained = False)

In [3]:
model

FCN(
  (backbone): IntermediateLayerGetter(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequenti

In [4]:
model.classifier[4] = nn.Conv2d(512, 1, kernel_size=(1, 1), stride=(1, 1))

In [5]:
img = torch.randn(2, 3, 512, 512)
pred = model(img)

torch.Size([2, 2048, 64, 64])
torch.Size([2, 1, 64, 64])
torch.Size([2, 1, 512, 512])
torch.Size([2, 1, 512, 512])
